<a href="https://colab.research.google.com/github/vuhpham94/nflx-data-projects-g6/blob/kwabena_branch/project2/notebooks/project2_g6_elnino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# drive connection
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restric

In [2]:
 #import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/vuhpham94/nflx-data-projects-g6/main/project2/resources/dataset/elnino.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("elnino.csv"), sep=",", header=True)

In [5]:
df.show()

+-----------+-----+------+----+------+---------+----------+------------+-----------------+---------+---------+-----------------+
|Observation| Year| Month| Day|  Date| Latitude| Longitude| Zonal Winds| Meridional Winds| Humidity| Air Temp| Sea Surface Temp|
+-----------+-----+------+----+------+---------+----------+------------+-----------------+---------+---------+-----------------+
|          1|   80|     3|   7|800307|    -0.02|   -109.46|        -6.8|              0.7|        .|    26.14|            26.24|
|          2|   80|     3|   8|800308|    -0.02|   -109.46|        -4.9|              1.1|        .|    25.66|            25.97|
|          3|   80|     3|   9|800309|    -0.02|   -109.46|        -4.5|              2.2|        .|    25.69|            25.28|
|          4|   80|     3|  10|800310|    -0.02|   -109.46|        -3.8|              1.9|        .|    25.57|            24.31|
|          5|   80|     3|  11|800311|    -0.02|   -109.46|        -4.2|              1.5|       

In [4]:
df.summary().show()

+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-----------------+------------------+------------------+
|summary|      Observation|              Year|            Month|               Day|              Date|          Latitude|         Longitude|        Zonal Winds|   Meridional Winds|         Humidity|          Air Temp|  Sea Surface Temp|
+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-----------------+------------------+------------------+
|  count|           178080|            178080|           178080|            178080|            178080|            178080|            178080|             178080|             178080|           178080|            178080|            178080|
|   mean|          89040.5| 93.30232479784367| 6.504

In [ ]:
df.printSchema()